In [1]:
using RigidBodyDynamics
using StaticArrays
using MeshCat, MeshCatMechanisms, Blink
using Random
Random.seed!(42);

WebIO._IJuliaInit()

In [2]:
srcdir = dirname(pathof(RigidBodyDynamics))
urdf = "../../../kortex_description/arms/gen3/7dof/urdf/GEN3_URDF_V12 copy.urdf"
mechanism = parse_urdf(urdf)
state = MechanismState(mechanism)
print(typeof(state))
mechanism

MechanismState{Float64, Float64, Float64, TypeSortedCollections.TypeSortedCollection{Tuple{Vector{Joint{Float64, Revolute{Float64}}}}, 1}}

Spanning tree:
Vertex: world (root)
  Vertex: Shoulder_Link, Edge: Actuator1
    Vertex: HalfArm1_Link, Edge: Actuator2
      Vertex: HalfArm2_Link, Edge: Actuator3
        Vertex: ForeArm_Link, Edge: Actuator4
          Vertex: SphericalWrist1_Link, Edge: Actuator5
            Vertex: SphericalWrist2_Link, Edge: Actuator6
              Vertex: Bracelet_Link, Edge: Actuator7
No non-tree joints.

In [3]:
body = findbody(mechanism, "Bracelet_Link")
point = Point3D(default_frame(body), 0., 0, -0.07)
point.v

3-element SVector{3, Float64} with indices SOneTo(3):
  0.0
  0.0
 -0.07

In [4]:
# Create the visualizer
vis = MechanismVisualizer(mechanism, URDFVisuals(urdf))

# Render our target point attached to the robot as a sphere with radius 0.07
setelement!(vis, point, 0.01)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat /home/amkyu/.julia/packages/MeshCat/Ax8pH/src/visualizer.jl:73


MeshCat Visualizer with path /meshcat/world/Shoulder_Link/HalfArm1_Link/HalfArm2_Link/ForeArm_Link/SphericalWrist1_Link/SphericalWrist2_Link/Bracelet_Link/after_Actuator7/<element> at http://127.0.0.1:8700

In [5]:
open(vis, Window());
mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf))

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8701
└ @ MeshCat /home/amkyu/.julia/packages/MeshCat/Ax8pH/src/visualizer.jl:73


MechanismVisualizer{MechanismState{Float64, Float64, Float64, TypeSortedCollections.TypeSortedCollection{Tuple{Vector{Joint{Float64, Revolute{Float64}}}}, 1}}, Visualizer}(MechanismState{Float64, Float64, Float64, …}(…), MeshCat Visualizer with path /meshcat at http://127.0.0.1:8701, 11)

In [6]:
function jacobian_transpose_ik!(state::MechanismState,
                               body::RigidBody,
                               point::Point3D,
                               desired::Point3D;
                               α=0.1,
                               iterations=100)
    mechanism = state.mechanism
    world = root_frame(mechanism)

    # Compute the joint path from world to our target body
    p = path(mechanism, root_body(mechanism), body)
    # Allocate the point jacobian (we'll update this in-place later)
    Jp = point_jacobian(state, p, transform(state, point, world))

    q = copy(configuration(state))

    for i in 1:iterations
        # Update the position of the point
        point_in_world = transform(state, point, world)
        # Update the point's jacobian
        point_jacobian!(Jp, state, p, point_in_world)
        # Compute an update in joint coordinates using the jacobian transpose
        Δq = α * Array(Jp)' * (transform(state, desired, world) - point_in_world).v
        # Apply the update
        q .= configuration(state) .+ Δq
        set_configuration!(state, q)
    end
    state
end

jacobian_transpose_ik! (generic function with 1 method)

In [21]:
# rand!(state)
# print(configuration(state))
# set_configuration!(vis, configuration(state))
q = [-0.0003347640156472966, 0.26032000097276037, 3.1400314573567463, -2.2697848286158764, -0.0004199120949869536, 0.9598967552397675, 1.5703141213719296]
set_configuration!(vis, q)


In [13]:
desired_tip_location = Point3D(root_frame(mechanism), 0.3, 0.3, 0.5)

Point3D in "world": [0.3, 0.3, 0.5]

In [12]:
jacobian_transpose_ik!(state, body, point, desired_tip_location)
set_configuration!(vis, configuration(state))

world = root_frame(mechanism)
p = path(mechanism, root_body(mechanism), body)
Jp = point_jacobian(state, p, transform(state, point, world))
print(Jp)

PointJacobian{Matrix{Float64}}(CartesianFrame3D: "world" (id = 0), [0.40992609325670437 -0.08567604899296258 0.2728775566145172 0.03791359726140542 0.02600476390765355 0.11195558471616357 5.551115123125783e-17; -0.43991088436868203 -0.24200280675155855 -0.37742576293685987 0.17068652238569607 -0.06121445495831368 -0.02109464994389182 -1.1102230246251565e-16; -3.2317658110280337e-6 0.5332340533956139 0.11752563030586596 0.44403507670252224 -0.03176672348881884 0.13406091038942114 -2.0816681711721685e-17])

In [16]:
transform(state, point, root_frame(mechanism))

Point3D in "world": [0.267226429562818, -0.27825725397822937, 0.581469173847727]

In [17]:
qs = typeof(configuration(state))[]
N = 61
ps = zeros(3, N)
for x in 0.515*ones(N)#0:0.0005:(N-1)*0.0005
    print(x)
    desired = Point3D(root_frame(mechanism), x, 0.435, 0.460)
    jacobian_transpose_ik!(state, body, point, desired)
    push!(qs, copy(configuration(state)))
end
ts = collect(0:0.05:(N-1)*0.05)


0.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.5150.515

61-element Vector{Float64}:
 0.0
 0.05
 0.1
 0.15
 0.2
 0.25
 0.3
 0.35
 0.4
 0.45
 0.5
 0.55
 0.6
 ⋮
 2.45
 2.5
 2.55
 2.6
 2.65
 2.7
 2.75
 2.8
 2.85
 2.9
 2.95
 3.0

In [20]:
setanimation!(vis, Animation(vis, ts, qs))

In [13]:
xs = Float64[]
ys = Float64[]
zs = Float64[]
# Downsample by 100 just so the plot doesn't become a huge file:
traj = zeros(32, N)
i = 1
for q in qs
#     print(q)
    traj[1:7,i] = q
    i = i + 1
    set_configuration!(state, q)
    p = transform(state, point, root_frame(mechanism))
    println(relative_transform(state, root_frame(mechanism), default_frame(body)))
    println()
    push!(xs, p.v[1])
    push!(ys, p.v[2])
    push!(zs, p.v[3])
end
traj[15,:] = xs
traj[16,:] = ys
traj[17,:] = zs

for i = 2:60
    traj[8:14, i] = (traj[1:7,i] - traj[1:7,i-1])/0.05
    traj[8:14, i] = (traj[1:7,i] - traj[1:7,i-1])/0.05
end

print(traj)

Transform3D from "world" to "after_Actuator7":
rotation: 2.986480108043739 rad about [-0.3472330549683346, -0.4712044315960144, 0.810799351986442], translation: [0.21045043762014173, 0.6623761386267449, 0.2571588420327006]

Transform3D from "world" to "after_Actuator7":
rotation: 3.005007431641143 rad about [-0.31496243773392313, -0.4664873791177766, 0.8265519874397088], translation: [0.1973983363373567, 0.6615420693577587, 0.1931824468873014]

Transform3D from "world" to "after_Actuator7":
rotation: 3.0153524574937474 rad about [-0.2968801629088846, -0.4642128384559141, 0.8344870337422251], translation: [0.18975179668911846, 0.6589074282886948, 0.16042829298475175]

Transform3D from "world" to "after_Actuator7":
rotation: 3.0209357896522517 rad about [-0.28662103453604443, -0.4637846843437385, 0.8383031367767071], translation: [0.18525033013765918, 0.6573177770776079, 0.14399221881637675]

Transform3D from "world" to "after_Actuator7":
rotation: 3.0242491491764456 rad about [-0.281001

rotation: 3.0613224405298953 rad about [-0.28768651808198337, -0.456861654481693, 0.8417326748903803], translation: [0.18132835011558263, 0.6549400648628758, 0.12643056968121566]

Transform3D from "world" to "after_Actuator7":
rotation: 3.0621469193602424 rad about [-0.28803827160252576, -0.45668149513009826, 0.8417101437537555], translation: [0.18137876723183446, 0.654931145578108, 0.12647012657831136]

Transform3D from "world" to "after_Actuator7":
rotation: 3.0629709330109387 rad about [-0.28839042045859753, -0.456501237487351, 0.8416873443032338], translation: [0.1814294065197985, 0.6549223087189919, 0.12651045279081588]

Transform3D from "world" to "after_Actuator7":
rotation: 3.0637944784064857 rad about [-0.28874296473991307, -0.4563208810164564, 0.8416642762180003], translation: [0.1814802682010609, 0.6549135538783954, 0.12655154883372563]

Transform3D from "world" to "after_Actuator7":
rotation: 3.0646175524684143 rad about [-0.28909590453013967, -0.4561404251809573, 0.8416409

[-2.205357273089756 -2.2634449370836007 -2.2913443934931053 -2.3056390590182443 -2.3129687429234322 -2.3166540653177834 -2.3183969322312974 -2.319090515147568 -2.3192123498058868 -2.3190209131168724 -2.3186571728501337 -2.3181983576752425 -2.3176868915073743 -2.317146117709061 -2.3165888944572743 -2.3160223095577757 -2.3154502720764394 -2.3148749387809318 -2.314297499953496 -2.313718612566497 -2.313138639190358 -2.3125577798152683 -2.3119761446149476 -2.311393794126217 -2.310810761443825 -2.3102270644850105 -2.3096427127690924 -2.3090577111663673 -2.30847206197188 -2.3078857660529386 -2.307298823484309 -2.306711233899955 -2.306122996687876 -2.3055341110980714 -2.3049445763023835 -2.304354391427693 -2.3037635555743314 -2.3031720678262997 -2.3025799272569514 -2.3019871329321413 -2.3013936839119915 -2.30079957925187 -2.300204818002947 -2.2996093992124957 -2.2990133219240834 -2.2984165851776646 -2.2978191880096555 -2.297221129452969 -2.29662240853705 -2.296023024287892 -2.295422975728058 -